Данный блокнот содержит все имеющиеся на текущий момент сведения по работе над датасетом первых воспоминаний 'reminiscence'.

Черновой план дальнейшей работы:
- Было бы удобно, если бы исходный датафрейм был глобальной переменной
- Преобразовать отображение чисел в возрастах к единообразному
- Инструмент для представления исходных данных и обработанных в верстке удобной для публикации в pdf (LaTeX), в виде статьи на Medium, в fb2, epub-форматах с заголовками и содержанием
- Сделать переводную версию
- Преобразование в pandas строковых значений чисел в числовые
- Инструмент перечисления номеров воспоминаний, в которых не заполнено переданное поле
- Инструмент оценки полноты датафрейма с рекомендациями работы над датафреймом
- Распределения записей по числовым параметрам:
    - Число символов
    - Число слов
    - Число воспоминаний
    - Возраст (общий интструмент для разных граф - с целью построения корреляций). Для пары "возраст-воспоминание" имеет смысл построить распределения самых ранних воспоминаний, если указан диапазон - то указывать среднюю точку с интервалом (необходимо найти подходящий тип визуализации)
    - Активное-пассивное
    - Доли воспоминаний с упоминанием параметра (цвет, ...)
    - Соотношение между полами
    - Наличие/отсутствие лиц и вообще инструмент заполненности поля (сколько символов "-" и сколько заполненных)
- Инструменты визуализации статистических распределений
- Использовать https://github.com/natasha/natasha и pymorphy
- pymorphy: какое время используется у глаголов, частотное распределение слов, приведенных к нормальной форме, разбивать составные воспоминания по отступ в виде одной строки или специальному символу. Отношение между частями речи. Что преобладает в коротких воспоминаниях? Анализ по парам младший/старший, мама/пара, бабушка/дедушка.
- pymorhy:
    - Выделение в нормальной форме всех частей речи, построение распределений для каждой части речи по общему массиву
    - Выделение всех корней
- Сопоставление отец-папа, мама и папа-родители (совместность).
- Продумать процесс работы с ячейками перевода воспоминаний
- Функция генерации текста из чередующихся "малых" и "крупных" воспоминаний для лучшего темпа представления в виде текста

# Считывание данных из Google Sheet

Набор данных оформлен [в виде электронной таблицы на Google Drive](https://docs.google.com/spreadsheets/d/1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI/edit?usp=sharing). Доступ к Google Sheets API осуществляется в соответствии с [документацией API для Python](https://developers.google.com/sheets/api/quickstart/python).

Импортируем данные учетной записи и преобразуем в объект Pandas:

In [1]:
import json
import os.path
import pickle       # для хранения токенов
import pandas as pd
    
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI' # id гугл-таблицы
RANGE_NAME = 'Memories' # Забираем лист целиком диапазон

In [2]:
def get_data():
    """Авторизует и забирает данные из Google SpreadSheet"""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    
    return values

## Обработка данных в Pandas

Создаем датафрейм. Удаляем те строки, что не относятся к датасету (комментарии в конце файла)

In [3]:
def get_df():
    data = get_data()
    df = pd.DataFrame.from_records(data)
    headers = df.iloc[0]  # Названия столбцов содержатся в нулевой строке таблицы
    df = pd.DataFrame(df.values[1:], columns=headers)
    df = df.set_index('№') # Номер строки соответствует номеру воспоминания
    
    # Пока не рассматриваются переводные записи. Комментарии используются лишь как заметки
    df = df.drop(columns=['Name', 'Translation', 'Translation footnote', 'Комментарий'])
    
    # Добавим столбец подсчета числа символов, из которых состоит запись.
    df ['Число символов'] = df['Воспоминание'].apply(lambda x: len(x))
    
    return df

Для обзора датасета создадим функции подсчета числа пустых ячеек определенного поля, а также вывода необработанной части датасета в порядке возрастания объема записи.

In [6]:
def col_empty_cells(df, col_name:str):
    """Находит пустые и NoneType ячейки столбца с именем col_name"""
    none_cells = df[col_name][~df[col_name].notna()].index
    empty_cells = df[col_name][df[col_name]==''].index
    ids = sorted(list(set(none_cells) | set(empty_cells)))
    return ids


def percent_line():
    """Определяет долю полностью обработанных записей, не содержащие пустых строк во всех полях,
    за исключением графы Комментарий. Выводятся строки, в порядке возрастания объема воспоминания"""
    # отбираем записи, содержащие не более 1 пустой графы - обычно это графа комментарий 
    df = get_df()
    df_not_ready = df[df.apply(lambda x: x.isnull().sum(), axis='columns') != 0]
    p = 100*(1-len(df_not_ready)/len(df))
    print("Число необработанных записей {0}, подготовлены {1:.1f}% датасета.".format(len(df_not_ready), p))
    return df_not_ready.sort_values(by='Число символов').head(10)

In [8]:
percent_line()

Число необработанных записей 226, это 65.0% датасета.


,Имя,Воспоминание,Пол,Кол-во,Возраст,Ориентир возраста,Мотив,Лица,Предметы,Цвета,Звуки,"Запахи, вкус",Другие ощущения,Сущности,Пространство,Время дня,Время года,Поведение,Число символов
№,,,,,,,,,,,,,,,,,,,
476,Майк (финн),,м,,3-4,None,None,None,None,None,None,None,None,None,None,None,None,None,0
487,Алена,"Как родители закрылись на кухне, и что-то долг...",ж,1,2-2.5,,,"папа, мама",None,None,None,None,None,None,None,None,None,None,103
386,-,"Маленький, почти секундный эпизод в возрасте т...",ж,1,3,None,None,None,None,None,None,None,None,None,None,None,None,None,106
378,-,Моя новорождённая сестра в роддоме в каком-то ...,ж,1,4,None,None,None,None,None,None,None,None,None,None,None,None,None,111
414,Муза,Запуск Спутника. Мне было полтора года. Все вы...,ж,1,1.5,None,None,None,None,None,None,None,None,None,None,None,None,None,111
454,Валера,Лет пять. Зима. Я в Нижнем Новгороде стою на м...,м,1,5,,,,,,,,,,,,зима,None,114
444,Матвей,Я помню себя еще в утробе и как рождался. Моме...,м,1,0,None,None,None,None,None,None,None,None,None,None,None,None,None,115
452,Маша,"Я выхожу из дома с няней, мы едем в зоопарк, у...",ж,1,3-4,None,None,None,None,None,None,None,None,None,None,None,None,None,116
352,Алеся,Около трех с половиной лет. Я вышла во двор с ...,ж,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,116


# Исследование распределений

Полностью заполнена лишь графа, содержащая текст воспоминания. Для указания того, что в тексте воспоминания не имеется дополнительной информации, указывает знак "-" (минус). Для анализа полноты используется функция `minus_or_smth_dstrb`.

In [273]:
def minus_or_smth(col_name: str):
    """Выводит число пустых ячеек, ячеек с символом минус и ячеек, содержащих информацию"""
    df = get_df()
    empty_cell_ids = col_empty_cells(df, col_name)
    print('Число пустых ячеек: {}.'.format(len(empty_cell_ids)))
    df_essential = df.drop(empty_cell_ids)
    column_essential = df_essential[col_name]
    minus_cell_ids = column_essential[column_essential=='-'] 
    print('Число записей, не содержащих информации про {}: {}.'.format(col_name.lower(), len(minus_cell_ids)))
    plus_cell_ids = column_essential[column_essential!='-']
    print('Число записей, содержащих информацию про {}: {}.'.format(col_name.lower(), len(plus_cell_ids)))
    return empty_cell_ids, minus_cell_ids, plus_cell_ids

## Распределение возраста

Возраста приводятся в форме приблизительного или точного (при соотнесении с событием) возраста, либо интервала. Интервалы в формате `начало диапазона-конец диапазона` приводим в форму двух чисел. Также имеются описания в которых используются не отрезки, а "лучи" времени (например, "не позже пяти", "после трех лет"). Все виды записей возраста представляются в виде списка словарей с ключами `float`, `range`, `more`, `less`.

In [290]:
def age(age_str:'str'):
    '''Возвращает список разбитых на категории возрастов воспоминаний'''
    list_of_ages = []
    
    def age_float(s:'str'):
        list_of_ages.append({'float': float(s)})
    
    def age_range(s:'str'):
        r = [float(item) for item in s.split('-')]
        list_of_ages.append({'range': r})
        
    def age_not_single(s:str):
        if '-' in s:
            age_range(s)
        elif '<' in s:
            list_of_ages.append({'less': float(s[1:])})
        elif '>' in s:
            list_of_ages.append({'more': float(s[1:])})
        else:
            age_float(s)
    
    try:
        age_float(age_str)
    except ValueError:  #!! здесь блоки лучше заменить рекурсией или вызовом функции
        if ';' in age_str:
            for s in age_str.split(';'):
                age_not_single(s)
        else:
            age_not_single(age_str)
    
    return list_of_ages

In [305]:
df = get_df()
age_df = df.loc[minus_or_smth('Возраст')[2].index]
age_df['Возраст'].apply(lambda x: age(x))

Число пустых ячеек: 83.
Число записей, не содержащих информации про возраст: 61.
Число записей, содержащих информацию про возраст: 501.


№
1                             [{'float': 4.0}]
2                             [{'float': 1.5}]
3                      [{'range': [1.0, 3.0]}]
5                              [{'less': 7.0}]
6                             [{'float': 1.0}]
9                              [{'less': 1.0}]
10                            [{'float': 1.0}]
11                            [{'float': 3.0}]
12                     [{'range': [4.0, 5.0]}]
13                            [{'float': 3.0}]
14                            [{'float': 1.0}]
15                             [{'less': 1.0}]
18                            [{'float': 1.0}]
19                            [{'float': 5.0}]
21                            [{'float': 3.0}]
22                            [{'float': 3.0}]
25                             [{'less': 1.0}]
26                            [{'float': 3.0}]
27                            [{'float': 1.5}]
28                            [{'float': 1.0}]
29                            [{'float': 4.0}]
30         